In [10]:
"""Errors might remain if Kernel is not restarted"""
"""Changes to local modules might not update if Kernel is not restarted"""

'Changes to local modules might not update if Kernel is not restarted'

In [13]:
import numpy as np
# provide point coordinates/sets of points, 
# can be skipped if you already have a Euclidean distance matrix corresponding to your sets
M = 3
N = 5
X = np.random.uniform(size = (M, 4))
Y = np.random.uniform(size = (N, 3))

In [14]:
import scipy as sp
# compute distance/proximity within sets of points (must be Euclidean distance matrices) 
DX = sp.spatial.distance.pdist(X)
DX = sp.spatial.distance.squareform(DX) 
DY = sp.spatial.distance.pdist(Y)
DY = sp.spatial.distance.squareform(DY) 

In [15]:
# Optional: provide a distance vectors for distance to the origin 
UX = np.random.uniform(size = M)
UY = np.random.uniform(size = N)

In [16]:
# provide a proximity set matrix for between the points of X and Y satisfying condition (ii) of Theorem 2.1
# f cannot be zero everywhere
bxy0 = min(min(min(UX), min(UY)), min(np.min(DX[DX!=0]), np.min(DY[DY!=0])))
fXY = np.random.uniform(low = 0.5*bxy0, high = bxy0, size = (M, N))
#fXY = np.random.uniform(size = (M,N)) # an example where (ii) is not necessary 

In [17]:
# Provide parameters (c1, c2, c3) as dictionary, using conditions (i), (ii), and (iii) of Theorem 3.1 as a guideline
c1, c2 = 1/2, 2 
a = 1. - 1./(M+N)
b = 2.*c2/(M+N)
c3 =  min(((2.*c1 + c2) - b)/a, 2*c2+2)
c = {"c1":c1, "c2":c2, "c3":c3} 
print(c1, c2, c3) # make sure they are all positive

0.5 2 2.857142857142857


In [18]:
# Compute cosine law matrix
import Methods.CosLM as CosLM
COS_MAT, c1, c2, c3, zeta_f = CosLM.CosLM(DX, DY, UX, UY, fXY, c) # w1 = x1
print("Is the cosine law matrix is symmetric:", np.all(np.isclose(COS_MAT,COS_MAT.T)))
sigma, U = sp.linalg.eigh(COS_MAT)
sigma = np.real(sigma) # if COS_MAT is symmetric, thus imaginary number are supposed to be zero or numerical zeros
sigma[np.isclose(sigma, np.zeros(len(sigma)))] = 0
print(sigma[np.argsort(sigma)[::-1]])

Is the cosine law matrix is symmetric: True
[19.16342532  4.60580032  4.53934219  4.457644    4.31460156  4.23584051
  4.06939285  3.46006507  1.31391577  0.        ]


In [19]:
print("Is COS_MAT PSD:", np.all(sigma>=0))
if (not np.all(sigma>=0)):
    uf1 = np.max((UY**2)[np.newaxis, :] - fXY**2)
    uf2 = np.max((UX**2)[:, np.newaxis] - fXY**2)
    print("2*c2*zeta_f - c3*zeta_f=%.5f, uf1 = %.5f, uf2= %.5f"%(2*c2*zeta_f - c3*zeta_f, uf1, uf2),"\n Check if 2*c2*zeta_f - c3.zeta_f >= max(uf1, uf2)", 2*c2*zeta_f - c3*zeta_f >= max(uf1, uf2)) 
    print("c1*zeta_f + c2*zeta_f - c3*zeta_f = %.5f, u(w1, ox) = %5.5f"%(c1*zeta_f + c2*zeta_f - c3*zeta_f, UX[0]), "\n Check if c1*zeta_f + c2*zeta_f - c3*zeta_f >= u(w1, ox)^2", c1*zeta_f + c2*zeta_f - c3*zeta_f >= UX[0]**2)
    print("c1*zeta_f/2"%(c1*zeta_f/2), "\n Check if c1*zeta_f/2 - (1/2)*(c1*zeta_f + c2*zeta_f - c3*zeta_f - u(w1, ox)**2)>=0", c1*zeta_f/2 - (1/2)*(c1*zeta_f + c2*zeta_f - c3*zeta_f - UX[0]**2)>0)
    print("Theorem 3.1 for more")
    

Is COS_MAT PSD: True


In [20]:
### check distances and proximities in Euclidean embedding ###
M = DX.shape[0]
W = U.dot(np.diag(np.sqrt(sigma)))
EmbX = W[1:M+1, :]
EmbY = W[M+2:, :]
o = W[M+1, :]
DMat = sp.spatial.distance.pdist(W[1:, :]) 
DMat = sp.spatial.distance.squareform(DMat)
# Within sets 
DXe = DMat[:M, :M]
DYe = DMat[M+1:, M+1:]

print("Check theory on distances for the set X", np.all(np.isclose(DXe, np.sqrt(DX**2 + c3*zeta_f) - np.sqrt(np.diag(c3*zeta_f*np.ones(DX.shape[0])))))) # diagonal elements of distance matrices are always 0s
print("Check theory on distances for the set Y", np.all(np.isclose(DYe, np.sqrt(DY**2 + c3*zeta_f) - np.sqrt(np.diag(c3*zeta_f*np.ones(DY.shape[0])))))) # diagonal elements of distance matrices are always 0s
print("Check theory on distance to orgin for the set X:", np.all(np.isclose(DMat[M, :M], np.sqrt(UX**2 + c3*zeta_f))))
print("Check theory on distance to orgin for the set Y:", np.all(np.isclose(DMat[M, M+1:], np.sqrt(UY**2 + c3*zeta_f))))

# Between sets
fXYe = DMat[:M, M+1:]
print("Check theory on distance between points of sets:", np.all(np.isclose(fXYe, np.sqrt(fXY**2 + c3*zeta_f))))


Check theory on distances for the set X True
Check theory on distances for the set Y True
Check theory on distance to orgin for the set X: True
Check theory on distance to orgin for the set Y: True
Check theory on distance between points of sets: True
